In [1]:
import gc
import pandas as pd

from scipy.sparse import csr_matrix, hstack

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

In [2]:
class_names = ['hindi', 'tamil','english']

In [3]:
input_val = [['mujhe choti se ek help chahie hei'],['home loan ka maximum tenure kya hai'],['Does your card offer any unique benefits'],['card epadi activate panradu']]

In [4]:
test = pd.DataFrame(input_val , columns= ['text'])

In [5]:
test.head()

,text
0,mujhe choti se ek help chahie hei
1,home loan ka maximum tenure kya hai
2,Does your card offer any unique benefits
3,card epadi activate panradu


In [6]:
# train = pd.read_csv('lang_detect_train.csv').fillna(' ')
# #test_test = pd.read_csv('lang_detect_test.csv').fillna(' ')

In [10]:
# train_text = train['text']
# #test_test_text = test_test['text']
test_text = test['text']
# all_text = pd.concat([train_text,test_text])


In [11]:
# word_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='word',
#     token_pattern=r'\w{1,}',
#     ngram_range=(1, 2),
#     max_features=50000)
# word_vectorizer.fit(all_text)
import pickle
tf1 = pickle.load(open("feature.pkl",'rb'))
test_word_features = tf1.transform(test_text)
print('Word TFIDF features loaded')

Word TFIDF features loaded


In [13]:
test_features = hstack([test_word_features])
print('HStack ')

HStack 


In [14]:
submission = pd.DataFrame.from_dict({'text': test['text']})
submission

,text
0,mujhe choti se ek help chahie hei
1,home loan ka maximum tenure kya hai
2,Does your card offer any unique benefits
3,card epadi activate panradu


In [15]:
for class_name in class_names:
    print(class_name)
    sfm = pickle.load(open("sfm"+class_name+".pkl",'rb'))
    test_sparse_matrix = sfm.transform(test_features)
    # Load from file
    joblib_file = 'lang_detect_lgbmodel_'+class_name+'.txt'
    model = lgb.Booster(model_file='{}'.format(joblib_file))
    print(model)
    submission[class_name] = model.predict(test_sparse_matrix,predict_disable_shape_check=True)

hindi
tamil
english


In [16]:
submission.head()

,text,hindi,tamil,english
0,mujhe choti se ek help chahie hei,0.976762,0.165112,0.018531
1,home loan ka maximum tenure kya hai,0.995934,0.103070,0.254609
2,Does your card offer any unique benefits,0.016150,0.294201,0.548872
3,card epadi activate panradu,0.016150,0.735124,0.126948
